In [8]:
import numpy as np
import pandas as pd
from collections import Counter

#树节点类，分为叶子节点，非叶子节点
class TreeNode(object):
    def __init__(self,is_leaf=False,score=None,
               feature=None,threshold=None,
               left=None,right=None):
        self.is_leaf=is_leaf
        self.score=score
        self.feature=feature
        self.threshold=threshold
        self.left=left
        self.right=right

In [9]:
#测试集只有单条的预测
def predict_single(treenode,test):
    if treenode.is_leaf:
         return treenode.score
    else:
        #测试集当前节点特征的值小于域值，向训练好的treenode左子树查找
        if test[treenode.feature]<treenode.threshold:
            return predict_single(treenode.left,test)
        else:
            return predict_single(treenode.right,test)

In [10]:
#测试集多条特征预测
def predict(treenode,test):
    result = []
    for i in test:
        result.append(predict_single(treenode,i))
    return result

In [11]:
#决策树类，基于Top-Down思路，
#predict->construct_tree->calculate_score->find_best_feature_threshold_and_gain
class Tree(object):
    #estimator为训练好的决策树
    def __init__(self,estimator=None,max_depth=3,
               min_sample_split=10,gamma=0):
        self.estimator=estimator
        self.max_depth=max_depth
        self.min_sample_split=min_sample_split
        self.gamma=gamma
        
    #测试集只有单条的预测
    def predict_single(self,treenode,test):
        if treenode.is_leaf:
            return treenode.score
        else:
            print(' return self.predict_single(treenode.left,test)')
            #测试集当前节点特征的值小于域值，向训练好的treenode左子树查找
            if test[treenode.feature]<treenode.threshold:
                return self.predict_single(treenode.left,test)
            else:
                return self.predict_single(treenode.right,test)
            
    #测试集多条特征预测
    def predict(self,test):
        result = []
        for i in test:
            result.append(predict_single(self.estimator,i))
        return result
    
    #构建，construct_tree方法构建实例变量训练决策树estimator
    def fit(self,train,label):
        self.estimator = self.construct_tree(train,label,depth_left = self.max_depth)
        
    def construct_tree(self,train,label,depth_left):
        #决策树终止条件,剩余树深=0或者节点中样本数<self.min_sample_split
        if depth_left==0 or len(train) < self.min_sample_split:
            return TreeNode(is_leaf=True,score=self.calculate_score(label))
        
        feature,threshold,gain = self.find_best_feature_threshold_and_gain(train,label)
        #增益<=最大增益
        if gain<=self.gamma:
            return TreeNode(is_leaf=True,score=self.calculate_score(label))
        
        index = train[:,feature]<threshold
        left = self.construct_tree(train[index],label[index],depth_left-1)
        right = self.construct_tree(train[~index],label[~index],depth_left-1)
        return TreeNode(feature=feature,threshold=threshold,left=left,right=right)
    
    #输入训练数据和标签，返回最佳分裂特征、最佳分裂点、增益
    def find_best_feature_threshold_and_gain(self,train,label):
        best_feature = None
        best_threshold = None
        best_gain = 0 
        for feature in range(train.shape[1]):
            print('feature:' + str(feature))
            threshold,gain = self.find_best_threshold_and_gain(train[:,feature],label)
            print(gain)
            if gain > best_gain:
                best_feature = feature
                best_threshold = threshold
                best_gain = gain
            print('best_feature:' + str(best_feature))
            print('best_threshold:' + str(best_threshold))
            print('best_gain:' + str(best_gain))
            print('--------------------------')
        return best_feature,best_threshold,best_gain
    
    #输入一个特征和对应标签，返回在这个特征上的最佳分裂点、增益
    def find_best_threshold_and_gain(self,feature_array,label):
        original_loss = self.calculate_loss(label)
        best_threshold = None
        best_gain = 0
        #去重并排序
        sorted_feature_values = np.unique(feature_array)
        #计算所有可能分裂点
        for i in range(1,len(sorted_feature_values)):
            threshold = (sorted_feature_values[i-1] + sorted_feature_values[i])/2
            index = feature_array < threshold
            left_loss = self.calculate_loss(label[index])
            right_loss = self.calculate_loss(label[~index])
            gain = original_loss - left_loss - right_loss
            print('threshold:' + str(threshold))
            print('gain=original_loss-left_loss1-right_loss='+
                  str(original_loss)+'-'+str(left_loss)+'-'+str(right_loss)+
                  '='+str(gain))
            if gain > best_gain:
                best_threshold = threshold
                best_gain = gain
            print('best_gain:' + str(best_gain))
            print('----------------------------------')    
        return best_threshold,best_gain
    
    #回归树:输入标签，返回损失函数的值,最小二乘法
    def calculate_loss(self,label):
        print(label)
        return np.sum(np.square(label-self.calculate_score(label)))
    
    #回归树:输入标签，返回预测值，平均值
    def calculate_score(self,label):
        #取平均值作为结果
        return np.mean(label)
    
    #分类树:输入标签，返回损失函数的值,最小交叉熵
    def calculate_loss(self,label):
        print(label)
        return np.sum(np.square(label-self.calculate_score(label)))
    
    #回归树:输入标签，返回预测值,概率最大值
    def calculate_score(self,label):
        #取平均值作为结果
        return np.mean(label)

In [12]:
train = np.array(
[
    [1,3],
    [1,1],
    [1,2],
    [1,2],
    [2,1],
    [3,2],
    [3,2],
    [3,1]
])
label =  np.array(
[6,3,2,3,4,2,2,2]
)

In [22]:
tree = Tree(max_depth=100,min_sample_split=1,gamma=0)
treenode = tree.construct_tree(train,label,3)
treenode.is_leaf

feature:0
[6 3 2 3 4 2 2 2]
[6 3 2 3]
[4 2 2 2]
threshold:1.5
gain=original_loss-left_loss1-right_loss=14.0-9.0-3.0=2.0
best_gain:2.0
----------------------------------
[6 3 2 3 4]
[2 2 2]
threshold:2.5
gain=original_loss-left_loss1-right_loss=14.0-9.2-0.0=4.800000000000001
best_gain:4.800000000000001
----------------------------------
4.800000000000001
best_feature:0
best_threshold:2.5
best_gain:4.800000000000001
--------------------------
feature:1
[6 3 2 3 4 2 2 2]
[3 4 2]
[6 2 3 2 2]
threshold:1.5
gain=original_loss-left_loss1-right_loss=14.0-2.0-12.0=0.0
best_gain:0
----------------------------------
[3 2 3 4 2 2 2]
[6]
threshold:2.5
gain=original_loss-left_loss1-right_loss=14.0-3.714285714285715-0.0=10.285714285714285
best_gain:10.285714285714285
----------------------------------
10.285714285714285
best_feature:1
best_threshold:2.5
best_gain:10.285714285714285
--------------------------
feature:0
[3 2 3 4 2 2 2]
[3 2 3]
[4 2 2 2]
threshold:1.5
gain=original_loss-left_loss1-right

False

In [19]:
tree.predict([[1,3],[3,1]])
tree.estimator.left.left.left.left.score

3.0

In [10]:
default = TreeNode(is_leaf=True,feature=None,threshold=None,left=None,right=None,score=1)
no_default = TreeNode(is_leaf=True,feature=None,threshold=None,left=None,right=None,score=0)
root = TreeNode(is_leaf=False,feature=0,threshold=16,left=None,right=None,score=None)
split1 = TreeNode(is_leaf=False,feature=1,threshold=5000,left=None,right=None,score=None)
split2 = TreeNode(is_leaf=False,feature=1,threshold=20000,left=None,right=None,score=None)
root.left = split2
root.right = split1
split1.left = default
split1.right = no_default
split2.left = default
split2.right = no_default

In [11]:
tree1=Tree(root)
tree1.predict([[16,18000],[1,1]])

[0, 1]

In [12]:
#分类树:输入标签，返回真实值
def realClass(label):
    class_set = np.unique(label)
    count = np.array([0]*class_set.size)
    #计算每种类别数量
    for i in label:
        for j in class_set:
            if i == j:
                index = np.where(class_set == j)[0]
                count[index] += 1      
    real_value = np.max(count)
    real_list = np.array([0]*class_set.size)
    index = count == np.max(count)
    return index.astype(int),count

In [13]:
label =  np.array(
[1,1,1,1,1,2,2,2,2,2,2,3]
)
realClass(label)

(array([0, 1, 0]), array([5, 6, 1]))